In [1]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

## Read Data 

In [ ]:
df_train = pd.read_csv('./titanic_data/train.csv')
df_test = pd.read_csv('./titanic_data/test.csv')

##  Profile Report
Is the same of Kaggle data description 

In [ ]:
#profile_report = ProfileReport(df_train)
#profile_report

# Baseline Model - Decision Tree Classifier 3 features

##### Transform string features into categorical number features

In [ ]:
df_train['Sex'] = preprocessing.LabelEncoder().fit_transform(df_train['Sex'])
df_train['Embarked'] = preprocessing.LabelEncoder().fit_transform(df_train['Embarked'])
df_test['Sex'] = preprocessing.LabelEncoder().fit_transform(df_test['Sex'])
df_test['Embarked'] = preprocessing.LabelEncoder().fit_transform(df_test['Embarked'])

In [ ]:
X = df_train[['Sex','Pclass','Embarked']]
y = df_train['Survived']

In [ ]:
clf = DecisionTreeClassifier(random_state=7)
scores = cross_val_score(clf, X, y, cv=10)
scores

array([0.82222222, 0.78651685, 0.78651685, 0.85393258, 0.86516854,
       0.79775281, 0.80898876, 0.76404494, 0.83146067, 0.79775281])

## Cose

In [ ]:
#fit the model with all train data

clf.fit(X,y)

#Predict over the test dataframe

predictions_baseline = clf.predict(df_test[['Sex','Pclass','Embarked']])

## Submissions

In [ ]:
#prepares a dataframe to be submitted

df_submission = pd.DataFrame({'PassengerId':df_test['PassengerId'], 'Survived':predictions_baseline})

#data types must be exactly the same as the training data

df_submission.Survived = df_submission.Survived.astype(int)

#only saves the two columns into the submission csv file

df_submission.to_csv( './results/baseline_submission.csv',index=False )

# Model 1 - Decision Tree Classifier all features

In [ ]:
mean_value=df_train['Age'].mean()

df_train['Sex'] = preprocessing.LabelEncoder().fit_transform(df_train['Sex'])
df_train['Embarked'] = preprocessing.LabelEncoder().fit_transform(df_train['Embarked'])
df_train['Age'].fillna(value=mean_value, inplace=True)
df_train['Age'] = df_train['Age'] - mean_value

X = df_train[['Sex','Embarked','Age','Pclass','SibSp','Parch']]
y = df_train['Survived']

#Same for test

df_test['Sex'] = preprocessing.LabelEncoder().fit_transform(df_test['Sex'])
df_test['Embarked'] = preprocessing.LabelEncoder().fit_transform(df_test['Embarked'])
df_test['Age'].fillna(value=mean_value, inplace=True)
df_test['Age'] = df_test['Age'] - mean_value
X_test = df_test[['Sex','Embarked','Age','Pclass','SibSp','Parch']]

X.info(True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Sex       891 non-null    int32  
 1   Embarked  891 non-null    int32  
 2   Age       891 non-null    float64
 3   Pclass    891 non-null    int64  
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
dtypes: float64(1), int32(2), int64(3)
memory usage: 34.9 KB


In [ ]:
clf = DecisionTreeClassifier(random_state=7)

clf.fit(X,y)

predictions_model_1 = clf.predict(X_test)

In [ ]:
#prepares a dataframe to be submitted

df_submission = pd.DataFrame({'PassengerId':df_test['PassengerId'], 'Survived':predictions_model_1})

#data types must be exactly the same as the training data

df_submission.Survived = df_submission.Survived.astype(int)

#only saves the two columns into the submission csv file

df_submission.to_csv( './results/model_1_submission.csv',index=False )

# Model 2 - Neural Network

In [13]:
import tensorflow as tf

In [14]:
mean_value=df_train['Age'].mean()

df_train['Sex'] = preprocessing.LabelEncoder().fit_transform(df_train['Sex'])
df_train['Embarked'] = preprocessing.LabelEncoder().fit_transform(df_train['Embarked'])
df_train['Age'].fillna(value=mean_value, inplace=True)
df_train['Age'] = df_train['Age'] - mean_value

X = df_train[['Sex','Embarked','Age','Pclass','SibSp','Parch']]
y = df_train['Survived']

#Same for test

df_test['Sex'] = preprocessing.LabelEncoder().fit_transform(df_test['Sex'])
df_test['Embarked'] = preprocessing.LabelEncoder().fit_transform(df_test['Embarked'])
df_test['Age'].fillna(value=mean_value, inplace=True)
df_test['Age'] = df_test['Age'] - mean_value
X_test = df_test[['Sex','Embarked','Age','Pclass','SibSp','Parch']]

In [15]:
X = X.to_numpy()
y = y.to_numpy()
X_test = X_test.to_numpy()

In [ ]:
model_2 = tf.keras.Sequential([
    tf.keras.layers.Dense(10000, input_shape = (6,)),
    tf.keras.layers.Dense(10000, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model_2.summary()
model_2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
history = model_2.fit(X, y, epochs=200)

In [29]:
test_predict = model_2.predict(X_test)

In [33]:
from sklearn.preprocessing import Binarizer
binarizer=Binarizer(threshold=0.5)
test_predict_result=binarizer.fit_transform(test_predict)
test_predict_result=test_predict_result.astype(np.int32)
test_predict_result=tf.squeeze(test_predict_result)

In [35]:
#prepares a dataframe to be submitted

df_submission = pd.DataFrame({'PassengerId':df_test['PassengerId'], 'Survived':test_predict_result})

#data types must be exactly the same as the training data

df_submission.Survived = df_submission.Survived.astype(int)

#only saves the two columns into the submission csv file

df_submission.to_csv( './nn_submission.csv',index=False )